<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MOM_Gradio_with_pdf_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dineshhn/MinutesOfMeeting/blob/updated_mom/MinutesOfMeeting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install all the dependencies. Place any new installables into requirements.txt file.
!pip install -r /content/drive/MyDrive/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import torch

In [4]:
!pip install datasets
from datasets import load_dataset
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Select an audio file and read it:
ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
audio_sample = ds[0]["audio"]

# Load the Whisper model with SDPA attention
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny.en")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-tiny.en", attn_implementation="sdpa")

# Enable static cache and compile the forward pass
model.generation_config.cache_implementation = "static"
model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

# Use the model and processor to transcribe the audio:
input_features = processor(
    audio_sample["array"], sampling_rate=audio_sample["sampling_rate"], return_tensors="pt"
).input_features

# Compile the forward pass
for _ in range(2):
    model.generate(input_features)

# Generate token ids using compiled graph (fast!)
predicted_ids = model.generate(input_features)

# Decode token ids to text
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

transcription[0]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/520 [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/9.19M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/73 [00:00<?, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.'

In [5]:
from transformers import AutoProcessor, Wav2Vec2BertForCTC
from datasets import load_dataset
import torch

dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation", trust_remote_code=True)
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")
model = Wav2Vec2BertForCTC.from_pretrained("hf-audio/wav2vec2-bert-CV16-en")

# audio file is decoded on the fly
inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")
with torch.no_grad():
    logits = model(**inputs).logits
predicted_ids = torch.argmax(logits, dim=-1)

# transcribe speech
transcription = processor.batch_decode(predicted_ids)
transcription[0]

inputs["labels"] = processor(text=dataset[0]["text"], return_tensors="pt").input_ids

# compute loss
loss = model(**inputs).loss
round(loss.item(), 2)

librispeech_asr_demo.py:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

preprocessor_config.json:   0%|          | 0.00/299 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/369 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/32.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.42G [00:00<?, ?B/s]

17.04

In [6]:
import torch
from transformers import pipeline

whisper = pipeline("automatic-speech-recognition", "openai/whisper-large-v3", torch_dtype=torch.float16, device="cuda:0")

# transcription = whisper("/content/drive/MyDrive/ColabNotebooks/EarningsCall.wav", return_timestamps=True)

# print(transcription["text"])

config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.90k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
!pip install deepspeech

ERROR: Could not find a version that satisfies the requirement deepspeech (from versions: none)
ERROR: No matching distribution found for deepspeech


In [8]:
# whisper-large-v3
def transcribe_audio_with_whisper(file):
    whisper = pipeline(
        "automatic-speech-recognition",
        "openai/whisper-large-v3",
        torch_dtype=torch.float16,
        device="cuda:0"
    )

    start_trans = time.time()
    transcription = whisper(file, return_timestamps=True)
    end_trans = time.time()
    full_transcription = transcription["text"]
    trans_time = f"Transcription time: {end_trans - start_trans:.2f} seconds"

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    start_summ = time.time()
    summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
    end_summ = time.time()
    summ_time = f"Summarization time: {end_summ - start_summ:.2f} seconds"

    return full_transcription, summary, trans_time, summ_time

In [9]:
# Wav2vec2
from transformers import pipeline
import torch
import time

def transcribe_audio_with_wav2vec2(file):
    # Initialize the Wav2Vec2 ASR model
    wav2vec2 = pipeline(
        "automatic-speech-recognition",
        model="facebook/wav2vec2-large-960h",
        torch_dtype=torch.float16,
        device="cuda:0"
    )

    start_trans = time.time()
    # Transcribe the audio with timestamps
    transcription = wav2vec2(file, return_timestamps=True)
    end_trans = time.time()
    full_transcription = transcription["text"]
    trans_time = f"Transcription time: {end_trans - start_trans:.2f} seconds"

    # Initialize the summarization model
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    start_summ = time.time()
    # Summarize the transcription
    summary = summarizer(full_transcription, max_length=130, min_length=30, do_sample=False)[0]["summary_text"]
    end_summ = time.time()
    summ_time = f"Summarization time: {end_summ - start_summ:.2f} seconds"

    return full_transcription, summary, trans_time, summ_time


In [10]:
# Dispatcher
def process_based_on_selection(file, choice):
    if choice == "Whisper-large-v3":
        return transcribe_audio_with_whisper(file)
    elif choice == "Facebook - Wav2vec2":
        result = transcribe_audio_with_wav2vec2(file)
    else:
        return "Invalid model", "Invalid model", "", ""

In [11]:
def extract_action_items(full_transcription, keywords):
    # Use default keywords if none provided
    if not keywords:
        keywords = [
            "must", "should", "need to", "investing", "expanding",
            "diversifying", "enhancing", "revenue", "risk", "loss"
        ]
    else:
        keywords = [kw.strip().lower() for kw in keywords.split(",")]

    # Split transcription into sentences
    sentences = re.split(r'(?<=[.!?])\s+', full_transcription)

    # Filter sentences containing any keyword
    action_items = [
        sentence for sentence in sentences
        if any(keyword in sentence.lower() for keyword in keywords)
    ]

    return action_items


In [12]:
def process_action_items(file, choice, keywords):
    transcription, _, _, _ = process_based_on_selection(file, choice)

    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."

    return f"{formatted_actions}\n\n{action_time}"


In [13]:
import nltk
import shutil
import os

# Clean corrupted tokenizer
nltk_data_path = "/usr/local/share/nltk_data/tokenizers/punkt"
if os.path.exists(nltk_data_path):
    shutil.rmtree(nltk_data_path)

# Redownload
nltk.download('punkt', force=True)
nltk.data.path.append("/usr/local/share/nltk_data")  # Ensure path is visible


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
import matplotlib.pyplot as plt
import numpy as np
import base64
from io import BytesIO
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from rouge_score import rouge_scorer

def format_metrics(metrics):
    """Format the metrics dictionary for better readability."""
    formatted = "\n".join([f"{key}: {value:.2f}" for key, value in metrics.items()])
    return formatted

def evaluate_summary(transcription, summary):
    # Tokenize the transcription and summary
    trans_tokens = transcription.split()
    summ_tokens = summary.split()

    # Generate binary vectors for token matching
    common_tokens = set(trans_tokens).union(set(summ_tokens))
    trans_vector = [1 if token in trans_tokens else 0 for token in common_tokens]
    summ_vector = [1 if token in summ_tokens else 0 for token in common_tokens]

    # Calculate evaluation metrics
    accuracy = accuracy_score(trans_vector, summ_vector)
    precision = precision_score(trans_vector, summ_vector, zero_division=0)
    recall = recall_score(trans_vector, summ_vector, zero_division=0)
    f1 = f1_score(trans_vector, summ_vector, zero_division=0)

    # ROUGE score calculation
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    rouge_scores = scorer.score(transcription, summary)
    rouge = rouge_scores['rougeL'].fmeasure

    metrics = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'ROUGE': rouge
    }

    # Convert metrics to lists for plotting
    metric_names = list(metrics.keys())
    metric_values = list(metrics.values())

    # Plotting the metrics
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))

    # Bar Plot
    axs[0, 0].bar(metric_names, metric_values, color='skyblue')
    axs[0, 0].set_title('Evaluation Metrics - Bar Plot')
    axs[0, 0].set_ylabel('Score')

    # Pie Chart
    axs[0, 1].pie(metric_values, labels=metric_names, autopct='%1.1f%%', startangle=90)
    axs[0, 1].set_title('Evaluation Metrics - Pie Chart')

    # Line Plot
    axs[1, 0].plot(metric_names, metric_values, marker='o', linestyle='-', color='blue')
    axs[1, 0].set_title('Evaluation Metrics - Line Plot')
    axs[1, 0].set_ylabel('Score')
    axs[1, 0].set_ylim(0, 1)

    # Radar Plot
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    values = metric_values + metric_values[:1]
    angles += angles[:1]

    axs[1, 1].set_title('Evaluation Metrics - Radar Plot', size=15, color='black', y=1.1)
    axs[1, 1].fill(angles, values, color='purple', alpha=0.25)
    axs[1, 1].plot(angles, values, color='purple', linewidth=2)
    axs[1, 1].set_yticks([])
    axs[1, 1].set_xticks(angles[:-1])
    axs[1, 1].set_xticklabels(metric_names)

    plt.tight_layout()

    # Save the plot to a base64 string
    buffer = BytesIO()
    plt.savefig(buffer, format='png')
    plt.close(fig)
    buffer.seek(0)
    image_base64 = base64.b64encode(buffer.getvalue()).decode('utf-8')

    # Return formatted metrics and the image
    return format_metrics(metrics), image_base64


In [15]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=c9d3610bc8fc2771e289b1eb27f5c68a51516e36a5a86e110b1eb90683828f1f
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


In [16]:
import os
from fpdf import FPDF
from io import BytesIO
from PIL import Image
import base64


def process_and_format_outputs(file, choice, keywords):
    import time
    import matplotlib.pyplot as plt

    print("=== PROCESS STARTED ===")

    # Transcription & Summary
    transcription, summary, trans_time, summ_time = process_based_on_selection(file, choice)
    print("Transcription and summary complete")

    trans_with_time = f"{transcription}\n\n{trans_time}"
    summ_with_time = f"{summary}\n\n{summ_time}"
    both_times = f"{trans_time}\n{summ_time}"

    # Action Items
    start_action = time.time()
    action_items = extract_action_items(transcription, keywords)
    end_action = time.time()
    action_time = f"Action item extraction time: {end_action - start_action:.2f} seconds"
    print("Action item extraction complete")

    if action_items:
        formatted_actions = "\n".join([f"{i+1}. {item}" for i, item in enumerate(action_items)])
    else:
        formatted_actions = "No action items found."
    actions_with_time = f"{formatted_actions}\n\n"

    # Evaluation + Plot
    try:
        print("Running evaluate_summary()...")
        evaluation_results, plot_base64 = evaluate_summary(transcription, summary)
        print("Evaluation successful")
        evaluation_plot_html = f"<img src='data:image/png;base64,{plot_base64}'/>"
    except Exception as e:
        evaluation_results = f"Error during evaluation: {str(e)}"
        evaluation_plot_html = ""
        plot_base64 = ""
        print("Evaluation failed:", str(e))

    # Export to .pdf file with plot
    try:
        pdf_filename = "mom_report.pdf"

        # Create the PDF file
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        pdf.cell(200, 10, txt="=== TRANSCRIPTION ===", ln=True, align="L")
        pdf.multi_cell(0, 10, transcription)
        pdf.cell(200, 10, txt=f"Time: {trans_time}", ln=True, align="L")

        pdf.cell(200, 10, txt="=== SUMMARY ===", ln=True, align="L")
        pdf.multi_cell(0, 10, summary)
        pdf.cell(200, 10, txt=f"Time: {summ_time}", ln=True, align="L")

        pdf.cell(200, 10, txt="=== ACTION ITEMS ===", ln=True, align="L")
        pdf.multi_cell(0, 10, formatted_actions)

        pdf.cell(200, 10, txt="=== EVALUATION METRICS ===", ln=True, align="L")
        pdf.multi_cell(0, 10, evaluation_results)

        # Add plot to PDF if available
        if plot_base64:
            img_data = base64.b64decode(plot_base64)
            image = Image.open(BytesIO(img_data))
            img_path = "plot.png"
            image.save(img_path)
            pdf.image(img_path, x=10, y=None, w=180)

        pdf.output(pdf_filename)

        export_path = pdf_filename
        print(f"Report saved to: {export_path}")
    except Exception as e:
        export_path = None
        print("Report export failed:", str(e))

    print("=== PROCESS COMPLETE ===")
    return (
        trans_with_time,
        summ_with_time,
        both_times,
        actions_with_time,
        evaluation_results,
        evaluation_plot_html,
        export_path
    )


In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
!pip install gradio
import gradio as gr
from transformers import pipeline
import torch
import time
import re

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 108.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.6 MB/s eta 0:00:00


In [19]:
import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


In [20]:
import nltk
import shutil
import os

# Clean corrupted tokenizer
nltk_data_path = "/usr/local/share/nltk_data/tokenizers/punkt"
if os.path.exists(nltk_data_path):
    shutil.rmtree(nltk_data_path)

# Redownload
nltk.download('punkt', force=True)
nltk.data.path.append("/usr/local/share/nltk_data")  # Ensure path is visible


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [21]:
# Gradio demo-1
!pip install gradio
import gradio as gr
from transformers import pipeline
import torch
import time
import re

import nltk

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')


with gr.Blocks() as demo:
    gr.Markdown("### Minutes of meeting")

    with gr.Row():
        with gr.Column(scale=1):
            file_input = gr.File(label="Upload your audio file")
            dropdown = gr.Dropdown(choices=["Whisper-large-v3", "Facebook - Wav2vec2", "Whisper-Small", "Mozzilla - Deepspeech", "Google - Speech Recognition"], label="Select Model")
            keyword_input = gr.Textbox(
                label="Custom Action Item Keywords (comma-separated)",
                placeholder="e.g. must, should, expanding, revenue"
            )
            #with gr.Row():
                # btn_trans = gr.Button("Transcribe the uploaded file")
                # btn_summ = gr.Button("Generate the summary")
            #btn_time = gr.Button("Show Both Timings")
            btn_action = gr.Button("Extract Custom Action Items")
            btn_all = gr.Button("Process All")
            output_plot = gr.HTML(label="Metric Plot")
            output_download = gr.File(label="Download Report")

        with gr.Column(scale=1):
            output_trans = gr.Textbox(label="Transcribed Text", lines=10)
            output_summ = gr.Textbox(label="Summary", lines=6)
            output_time = gr.Textbox(label="Processing time", lines=2)
            output_action = gr.Textbox(label="Action Items", lines=10)
            output_metrics = gr.Textbox(label="Evaluation Metrics", lines=10)



    # Individual buttons
    # btn_trans.click(
    #     fn=lambda file, choice: f"{process_based_on_selection(file, choice)[0]}\n\n{process_based_on_selection(file, choice)[2]}",
    #     inputs=[file_input, dropdown],
    #     outputs=output_trans
    # )

    # btn_summ.click(
    #     fn=lambda file, choice: f"{process_based_on_selection(file, choice)[1]}\n\n{process_based_on_selection(file, choice)[3]}",
    #     inputs=[file_input, dropdown],
    #     outputs=output_summ
    # )

    # btn_time.click(
    #     fn=lambda file, choice: f"{process_based_on_selection(file, choice)[2]}\n{process_based_on_selection(file, choice)[3]}",
    #     inputs=[file_input, dropdown],
    #     outputs=output_time
    # )

    btn_action.click(
        fn=process_action_items,
        inputs=[file_input, dropdown, keyword_input],
        outputs=output_action
    )

    # Combined "Process All" button
    btn_all.click(
        fn=process_and_format_outputs,
        inputs=[file_input, dropdown, keyword_input],
        outputs=[output_trans, output_summ, output_time, output_action, output_metrics, output_plot, output_download]
    )

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://eeaf8c5e90d646b134.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
